In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#%matplotlib notebook
import matplotlib
matplotlib.use('Agg')
%matplotlib inline


In [ ]:
import requests
import json
import numpy as np
import scipy as sp
import librosa
import math
from pathlib import Path 
from pprint import pprint
import os
import dotenv
from pprint import pprint

np.set_printoptions(edgeitems=25, linewidth=100) 

dotenv.load_dotenv(dotenv.find_dotenv(), override=True)
os.environ['PS_CACHE_SIZE'] = "0"

input_sound_dir = Path("~/Source/paper_pattern_machine_omp").expanduser()/ 'examples/input_basic'
output_dir = Path("~/Source/paper_pattern_machine_omp/examples/").expanduser()

# sound_names = [p.name for p in input_sound_dir.glob("*.mp3")]
sound_names = [
#  'accordion.mp3',
#  'angklung.mp3',
#  'gamelanperc.mp3',
#  'jentreng.mp3',
#  'kumi.mp3',
#  'notesweep.mp3',
#  'rocknegativland.mp3',
#  'smoothsweep.mp3',
 'suling.mp3',
#  'tarawangsa.mp3',
#  'thaiflute.mp3',
 'thumbpiano.mp3',
 'trumpetsolo.mp3',
#  'voiceatonal.mp3',
 'voiceguitar.mp3'
]
pprint(sound_names)

In [ ]:
import shutil
import seaborn as sns
sns.set()
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc
import warnings 
import librosa
from librosa import display as lrd
from librosa.display import specshow, cmap as lr_cmap
import seaborn as sns
from pattern_machine.sample import Sample, load_sample
from pattern_machine.autocorr import autocorr_featurize, autocorr_slice_featurize
from pattern_machine.highlights import highlights
from pattern_machine import render_grains
from pattern_machine.pattern_iter import iterize_kwargs
from pattern_machine.timed_stream import Affine
from pattern_machine.plots import multi_save_fig

def unitize(A):
    bottom, middle, top = np.percentile(A, q=[10, 50, 90])
    A = A / (top*1.1-bottom*1.1)
    return np.clip(A, a_min=-1, a_max=1)

def cross_render_test(
        source_name,
        target_name,
        mosaic_namer,
        display=False,
        clobber=False,
        **kwargs):

    source_base = ".".join(source_name.split(".")[:-1])
    target_base = ".".join(target_name.split(".")[:-1])
    mosaic_name = mosaic_namer(source_name, target_name, **kwargs)
    mosaic_base = ".".join(mosaic_name.split(".")[:-1])
    print("mosaic name", mosaic_name)
    if (output_dir/mosaic_name).exists() and not clobber:
        warnings.warn("Path exists: {}".format(output_dir/mosaic_name))

    # trim quiet sections from audio
    source_sample = highlights(load_sample((input_sound_dir/source_name).expanduser(), sr=44100, duration=30), top_db=20)
    target_sample = load_sample((input_sound_dir/target_name).expanduser(), sr=44100, duration=5)
    
    # we would like to simply copy but this won't resepect truncation
    #shutil.copy(source_sample, output_dir/target_name)
    target_sample.save(output_dir/target_name, format="s16")
    print("saved to", target_sample.path())

    cmap = lr_cmap([-2,0,2], robust=False, cmap_div='PuOr')
    hop=2048
    ds = 4 
    w = 86/25 # convert width in mm into printer inches 
    t_s = 100

    rc('font',
        family='serif',
        serif=['DejaVu Serif', 'Palatino']
    )
    rc('mathtext', fontset='cm')
    # rc('text', usetex=True)
    sns.set_style("whitegrid")
    sns.set_context("paper")
    
    fig = plt.figure()
    ax = plt.gca()
    print(target_base)
    target_c = autocorr_slice_featurize(
        target_sample,
        hop_length=hop,
        frame_length=4096,
        delay_step_size=ds,
        n_delays=128,
        window='cosine'
    )[:,:]
    xi_s = target_c[:, :].shape[1] +1
    xi_rg = ds * np.arange(xi_s)

    specshow(
        unitize(target_c[:t_s, :].T),
        sr=44100,
        hop_length=hop,
        y_axis='linear',
        x_axis='off',
        y_coords=xi_rg,
        cmap=cmap,
    );
    # ax.set_ylabel(r'$\xi$')
    ax.set_ylabel(r'')
    ax.set_xlabel(
        '',
#         r'Time',
#         family='serif',
#         fontsize=9,
    )
    ax.yaxis.set_major_locator(
        plt.NullLocator()
    #     plt.MaxNLocator(2)
    #     plt.FixedLocator((0, (xi_s-1)*ds/2))
    )
    ax.xaxis.set_major_formatter(
        plt.FuncFormatter(lambda v, n: r"${:.0f}s$".format(v)))
    ax.xaxis.set_major_locator(
        plt.NullLocator()
#         plt.MaxNLocator(4)
    #     plt.FixedLocator((0, 1, 2, 3, 4))
    )

    multi_save_fig(
        basename=output_dir/target_base,
        fig=fig,
        dpi=105,
        verbose=11,
        suffixes=('.png',),
    )
    if display:
        plt.show()
    plt.clf()
    fig = plt.figure()
    ax = plt.gca()

    mosaic_sample = render_grains.cross_render_sample_adaptive(
        target_sample=target_sample,
        source_sample=source_sample,
        **kwargs
    )
    mosaic_sample.suffix = ".mp3"
    mosaic_sample.normalize()
    mosaic_sample.save(output_dir/mosaic_name, format="s16")
    print("saved to", mosaic_sample.path())
    
    mosaic_c = autocorr_slice_featurize(
        mosaic_sample, 
        hop_length=hop,
        frame_length=4096,
        delay_step_size=ds,
        n_delays=128,
        window='cosine'
    )[:,:]
    

    spec = specshow(
        unitize(mosaic_c[:, :].T),
        sr=44100,
        hop_length=hop,
        y_axis='linear',
        x_axis='time',
        y_coords=xi_rg,
        cmap=cmap,
    );
    # ax.set_ylabel(r'$\xi$')
    ax.set_ylabel(r'')
    ax.set_xlabel(
        '',
#         r'Time',
#         family='serif',
#         fontsize=9,
    )
    ax.yaxis.set_major_locator(
        plt.NullLocator()
    #     plt.MaxNLocator(2)
    #     plt.FixedLocator((0, (xi_s-1)*ds/2))
    )
    ax.xaxis.set_major_formatter(
        plt.FuncFormatter(lambda v, n: r"${:.0f}s$".format(v)))
    ax.xaxis.set_major_locator(
        plt.NullLocator()
#         plt.MaxNLocator(4)
    #     plt.FixedLocator((0, 1, 2, 3, 4))
    )
    multi_save_fig(
        basename=output_dir/mosaic_base,
        dpi=105,
        verbose=11,
        suffixes=('.png',),
    )

    if display:
        plt.show()
    return {
        mosaic_base: mosaic_sample
    }


def mosaic_name_basic(
        extra_text="test",
        ):
    def mosaic_namer(
            source_name,
            target_name,
            **args):
        source_base = ".".join(str(source_name).split(".")[:-1])
        target_base = ".".join(str(target_name).split(".")[:-1])
        mosaic_base = "_".join([
            source_base,
            target_base,
            extra_text
        ])
        return mosaic_base + '.mp3'
    return mosaic_namer

In [ ]:
# sound_names
# rz = cross_render_sample_adaptive(
#     target_sample=target_sample,
#     source_sample=source_sample,
#     analysis_window='cosine',
#     synthesis_window='cosine',
#     verbose=10,
#     code_size=40,
#     hop_length=2048,
#     density=4.0,
#     frame_length=8192,
#     delay_step_size=4,
#     grain_jitter=0.01,  # proportionally offset grain centers from each other
#     n_delays=128,
#     source_anchor='center', 
#     dest_anchor='center',  
#     match_size=1,
#     n_starts=17,
#     progress=True,
# #     **args
# )

In [ ]:
from joblib import Parallel, delayed
from pattern_machine.timed_stream import Affine
from itertools import product 

mosaic_namer = mosaic_name_basic("autocorrstatic")
res = dict()

sound_pairs = ((source_name, target_name) for (source_name, target_name) in product(list(sound_names), list(sound_names)) if  source_name != target_name)

with Parallel(
        n_jobs=6,
        verbose=15) as pool:
    r = pool(delayed(cross_render_test)(
        source_name, target_name,
        mosaic_namer,
        clobber=True,
        code_size=40,
        n_start=17,
#         basis_size=1,
        density=2.0,
        hop_length=2048,
        frame_length=8192,
        delay_step_size=4,
        grain_jitter=0.01,  # offset grain centers
#         codebook_a=Affine(offset=0.1, mul=0.2),
#         codebook_c=0.9,
        source_anchor='center',
        dest_anchor='center',
        analysis_window='cosine',
        synthesis_window='cosine',
        verbose=0,
        pdb=False,
        n_delays=128,
        display=False,
        progress=True,
        seed=13) for source_name, target_name in sound_pairs)


## web 2.0


In [ ]:
from yattag import Doc,indent
import markdown
from datetime import datetime
from IPython.core.display import display, HTML
from itertools import product 

TAIL = "autocorr"

mosaic_namer = mosaic_name_basic(TAIL)
res = dict()

sound_pairs = (
    (source_name, target_name)
    for (source_name, target_name)
    in product(list(sound_names), list(sound_names))
    if source_name != target_name
)

def unitize(A):
    bottom, middle, top = np.percentile(A, q=[10, 50, 90])
    A = A / (top*1.1-bottom*1.1)
    return np.clip(A, a_min=-1, a_max=1)


info_string = markdown.markdown("""
# Autocorrelogram synthesis examples

Prepared for the paper
*Style transfer using sparse autocorrelograms*
under submission.
""")

timestamp_string = markdown.markdown(f"""<p class="datestamp">Last updated<time> 
    {datetime.utcnow().isoformat(timespec='seconds')}
</time></p>""")

credit_string = markdown.markdown("""
## Audio recording credits

Recordings are used with licence of their respective creators, and
may not be further redistributed without permission unless otherwise stated.
                                     
* `suling.mp3`: Mohammed Dimas Firmansyah
* `thumbpiano.mp3`: Winter Riddle
* `trumpetsolo.mp3`: [Mihai Sorohan](http://www.mihaisorohan.net/)
* `voiceguitar.mp3`: Emm Collins and James Nichols
* `smoothsweep.mp3`: Public domain
""")

doc, tag, text, line = Doc().ttl()
doc.asis('<!DOCTYPE html>')

with tag('html'):
    with tag('head'):
        line('style', """
table {
  border-collapse: collapse;
}
td img,th img{
  width:256px;
  display:block;
}
td,th.source, th.target {
  border: 1px solid black;
  padding: 5px;
}
th.source {
    background-color: red;
    border-bottom: 3px solid black;
}
th.target {
    background-color: blue;
    border-right: 3px solid black;
}
td {
    background-color: magenta;
    font-style: italic;
}
.datestamp {
    font-style: italic;
    font-size: 80%;
}
        """, type="text/css")
        line('title', "Autocorrelogram synthesis examples")
    with tag('body'):
        doc.asis(info_string)

        with tag('table'):
            with tag('thead'):
                with tag('tr'):
                    doc.stag('th')  # empty
                    doc.stag('th')  # empty
                    line(
                        'th',
                        'Source',
                        colspan=len(sound_names),
                        klass="metaheader source")

                with tag('tr'):
                    doc.stag('th', klass="null")  # empty
                    doc.stag('th', klass="null")  # empty
                    for source_name in sound_names:
                        source_name = Path(source_name)
                        print('sn', source_name)
                        with tag('th', scope='col', klass="source"):
                            line('p', str(source_name) + " (source)")
                            with tag('audio', 'controls'):
                                doc.stag(
                                    'source',
                                    src="./" + str(source_name),
                                    type='audio/mpeg',
                                    klass='examplecell'
                                )
                            doc.stag(
                                'img',
                                src="./" + str(source_name.with_suffix('.png'))
                            )

            with tag('tbody'):
                for i, target_name in enumerate(sound_names):
                    with tag('tr'):
                        if i==0:
                            # rowspan is brain hurtey
                            with tag('th', rowspan=len(sound_names), klass="metaheader target"):
                                line('p', 'Target')
                        with tag('th', scope='row', klass='target'):
                            line('p', str(target_name)+ " (target)")
                            with tag('audio', 'controls'):
                                doc.stag(
                                    'source',
                                    src="./" + str(target_name),
                                    type='audio/mpeg',
                                    klass='examplecell'
                                )
                            doc.stag(
                                'img',
                                src="./" + str(Path(target_name).with_suffix('.png')),
                            )
                        for source_name in sound_names:
                            with tag('td'):
                                if source_name == target_name: continue
                                mosaic_name = Path(mosaic_namer(
                                    source_name, target_name))

                                mosaic_sample = load_sample(output_dir/mosaic_name.expanduser(), sr=44100)
                                cmap = lr_cmap([-1,0,1], robust=False, cmap_div='PuOr')
                                hop=2048
                                ds = 4 
                                w = 86/25 # convert width in mm into printer inches 
                                t_s = 100

                                rc('font',
                                    family='serif',
                                    serif=['DejaVu Serif', 'Palatino']
                                )
                                rc('mathtext', fontset='cm')
                                # rc('text', usetex=True)
                                sns.set_style("whitegrid")
                                sns.set_context("paper")

                                fig = plt.figure()
                                ax = plt.gca()
                                print(target_name)
                                cspec = autocorr_slice_featurize(
                                    mosaic_sample,
                                    hop_length=hop,
                                    frame_length=4096,
                                    delay_step_size=ds,
                                    n_delays=128,
                                    window='cosine'
                                )[:,:]
                                xi_s = cspec[:, :].shape[1] +1
                                xi_rg = ds * np.arange(xi_s)

                                specshow(
                                    unitize(cspec[:t_s, :].T),
                                    sr=44100,
                                    hop_length=hop,
                                    y_axis='linear',
                                    x_axis='off',
                                    y_coords=xi_rg,
                                    cmap=cmap,
                                );
                                # ax.set_ylabel(r'$\xi$')
                                ax.set_ylabel(r'')
                                ax.set_xlabel(
                                    '',
                                #         r'Time',
                                #         family='serif',
                                #         fontsize=9,
                                )
                                ax.yaxis.set_major_locator(
                                    plt.NullLocator()
                                #     plt.MaxNLocator(2)
                                #     plt.FixedLocator((0, (xi_s-1)*ds/2))
                                )
                                ax.xaxis.set_major_formatter(
                                    plt.FuncFormatter(lambda v, n: r"${:.0f}s$".format(v)))
                                ax.xaxis.set_major_locator(
                                    plt.NullLocator()
                                #     plt.MaxNLocator(4)
                                #     plt.FixedLocator((0, 1, 2, 3, 4))
                                )

                                multi_save_fig(
                                    basename=output_dir/(mosaic_name.stem),
                                    fig=fig,
                                    dpi=105,
                                    verbose=11,
                                    suffixes=('.png',),
                                )
                                line('p', str(mosaic_name) + " (mosaic)")
                                with tag('audio', 'controls'):
                                    doc.stag(
                                        'source',
                                        src="./" + str(mosaic_name),
                                        type='audio/mpeg',
                                        klass='examplecell'
                                    )
                                doc.stag(
                                    'img',
                                    src="./" + str(mosaic_name.with_suffix('.png')),
                                )

    doc.asis(credit_string)            
    doc.asis(timestamp_string)            

html_string = indent(doc.getvalue())

with open(output_dir/'index_{}.html'.format(TAIL), 'w') as h:
    h.write(html_string)

display(HTML(html_string))


## old webpage

In [ ]:
from yattag import Doc,indent
import markdown
from datetime import datetime
from IPython.core.display import display, HTML

info_string = markdown.markdown("""
# Autocorrelogram synthesis examples

Prepared for the paper
*Style transfer using sparse autocorrelograms*
under submission.
""")

timestamp_string = markdown.markdown(f"""<p class="datestamp">Last updated<time> 
    {datetime.utcnow().isoformat(timespec='seconds')}
</time></p>""")

credit_string = markdown.markdown("""
## Audio recording credits

Recordings are used with licence of their respective creators, and
may not be further redistributd without permission unless otherwise stated.
                                     
* `suling.mp3`: Mohammed Dimas Firmansyah
* `thumbpiano.mp3`: Winter Riddle
* `trumpetsolo.mp3`: [Mihai Sorohan](http://www.mihaisorohan.net/)
* `voiceguitar.mp3`: Emm Collins and James Nichols
* `smoothsweep.mp3`: Public domain
""")


doc, tag, text, line = Doc().ttl()
doc.asis('<!DOCTYPE html>')

with tag('html'):
    with tag('head'):
        line('style', """
table {
  border-collapse: collapse;
}
td img,th img{
  width:256px;
  display:block;
}
td,th.source, th.target {
  border: 1px solid black;
  padding: 5px;
}
th.source {
    background-color: red;
    border-bottom: 3px solid black;
}
th.target {
    background-color: blue;
    border-right: 3px solid black;
}
td {
    background-color: magenta;
    font-style: italic;
}
.datestamp {
    font-style: italic;
    font-size: 80%;
}
        """, type="text/css")
        line('title', "Autocorrelogram synthesis examples")
    with tag('body'):
        doc.asis(info_string)

        with tag('table'):
            with tag('thead'):
                with tag('tr'):
                    doc.stag('th')  # empty
                    doc.stag('th')  # empty
                    line(
                        'th',
                        'Source',
                        colspan=len(sound_names),
                        klass="metaheader source")

                with tag('tr'):
                    doc.stag('th', klass="null")  # empty
                    doc.stag('th', klass="null")  # empty
                    for source_name in sound_names:
                        source_name = Path(source_name)
                        print('sn', source_name)
                        with tag('th', scope='col', klass="source"):
                            line('p', str(source_name) + " (source)")
                            with tag('audio', 'controls'):
                                doc.stag(
                                    'source',
                                    src="./" + str(source_name),
                                    type='audio/mpeg',
                                    klass='examplecell'
                                )
                            doc.stag(
                                'img',
                                src="./" + str(source_name.with_suffix('.png'))
                            )

            with tag('tbody'):
                for i, target_name in enumerate(sound_names):
                    with tag('tr'):
                        if i==0:
                            # rowspan is brain hurtey
                            with tag('th', rowspan=len(sound_names), klass="metaheader target"):
                                line('p', 'Target')
                        with tag('th', scope='row', klass='target'):
                            line('p', str(target_name)+ " (target)")
                            with tag('audio', 'controls'):
                                doc.stag(
                                    'source',
                                    src="./" + str(target_name),
                                    type='audio/mpeg',
                                    klass='examplecell'
                                )
                            doc.stag(
                                'img',
                                src="./" + str(Path(target_name).with_suffix('.png')),
                            )
                        for source_name in sound_names:
                            with tag('td'):
                                if source_name == target_name: continue
                                mosaic_name = Path(mosaic_namer(
                                    source_name, target_name))
                                line('p', str(mosaic_name) + " (mosaic)")
                                with tag('audio', 'controls'):
                                    doc.stag(
                                        'source',
                                        src="./" + str(mosaic_name),
                                        type='audio/mpeg',
                                        klass='examplecell'
                                    )
                                doc.stag(
                                    'img',
                                    src="./" + str(mosaic_name.with_suffix('.png')),
                                )

    doc.asis(credit_string)            
    doc.asis(timestamp_string)            

    
html_string = indent(doc.getvalue())

with open(output_dir/'index.html', 'w') as h:
    h.write(html_string)

display(HTML(html_string))


In [ ]:
# from joblib import Parallel, delayed
from pattern_machine.timed_stream import Affine

# with Parallel(
#         n_jobs=n_workers,
#         verbose=verbose*3) as pool:


mosaic_namer = mosaic_name_basic("test")
res = dict()
for source_name in sound_names:
    local_targets = list(sound_names)
    # local_targets.remove(source_name)
    for target_name in local_targets:
        print(source_name, target_name)
        r = cross_render_test(
            source_name,
            target_name,
            mosaic_namer,
            code_size=42,
            n_start=16,
            basis_size=1,
            density=4.0,
            hop_length=1024,
            frame_length=4096,
            delay_step_size=4,
            grain_jitter=0.01,  # offset grain centers
            codebook_a=Affine(offset=0.1, mul=2.0),
            codebook_c=0.9,
            source_anchor='center',
            dest_anchor='center',
            analysis_window='cosine',
            synthesis_window='cosine',
            verbose=0,
            pdb=False,
            n_delays=256,
            display=True,
            seed=13)
        res.update(r)


In [ ]:
# from IPython.core.display import display, HTML
# def show_audio_with_controls(file_path):
#     display(HTML("<audio controls><source src={} type='audio/mpeg'></audio>".format(file_path)))
    